## LLaMA 2 指令微调（Alpaca-Style on Dolly-15K Dataset)

示例代码关键训练要素：
- 使用 Dolly-15K 数据集，以 Alpaca 指令风格生成训练数据
- 以 4-bit（NF4）量化精度加载 `LLaMA 2-7B` 模型
- 使用 QLoRA 以 `bf16` 混合精度训练模型
- 使用 `HuggingFace TRL` 的 `SFTTrainer` 实现监督指令微调
- 使用 Flash Attention 快速注意力机制加速训练（需硬件支持）

In [ ]:
from datasets import load_dataset
from random import randrange

dataset = load_dataset("databricks/databricks-dolly-15k", split='train')

In [ ]:
dataset

In [ ]:
print(dataset[randrange(len(dataset))])

### 以 Alpaca-Style 格式化指令数据

`Alpaca-style` 格式：https://github.com/tatsu-lab/stanford_alpaca#data-release

In [1]:
from textwrap import dedent

def format_instruction(sample_data):
    """
    Formats the given data into a structured instruction format.

    Parameters:
    sample_data (dict): A dictionary containing 'response' and 'instruction' keys.

    Returns:
    str: A formatted string containing the instruction, input, and response.
    """
    if 'response' not in sample_data or 'instruction' not in sample_data:
        return "Error: 'response' or 'instruction' key missing in the input data."
    
    result = f"""
    ### Instruction:
    Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 

    ### Input:
    {sample_data['response']}

    ### Response:
    {sample_data['instruction']}
    """
    return dedent(result).strip()


In [ ]:
print(format_instruction(dataset[randrange(len(dataset))]))

### 使用快速注意力（Flash Attention）加速训练

检查你的 GPU 是否支持 `flash-attn` 加速：

```shell
$ python -c "import torch; assert torch.cuda.get_device_capability()[0] >= 8, 'Hardware not supported for Flash Attention'"

Traceback (most recent call last):
  File "<string>", line 1, in <module>
AssertionError: Hardware not supported for Flash Attention
```
**运行结果：演示使用的 NVIDIA T4 硬件不支持 Flash Attention**

#### 安装 flash-attn 加速包（需要GPU硬件支持）

```shell
$ MAX_JOBS=4 pip install flash-attn --no-build-isolation
```

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

use_flash_attention = False

if torch.cuda.get_device_capability()[0] >= 8:
    from utils.llama_patch import replace_attn_with_flash_attn
    print('Using flash attention')
    replace_attn_with_flash_attn()
    use_flash_attention = True

# 获取 LLaMA 2-7B 模型权重
# 无需 Meta AI 审核的模型权重
model_id = "NousResearch/Llama-2-7b-hf" 
# 通过 Meta AI 审核后可使用此 Model ID 下载
# model_id = "meta-llama/Llama-2-7b-hf"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type='nf4',
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,
    use_cache=False,
    device_map='auto')
model.config.pretraining_tp = 1

if use_flash_attention:
    from utils.llama_patch import forward
    assert model.model.layers[0].self_attn.forward.__doc__ == forward.__doc__, 'Model is not using flash attention'

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

In [ ]:
from peft import LoraConfig, prepare_model_for_kbit_training, get_peft_model

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=16,
    bias='none',
    task_type='CAUSAL_LM'
)

model = prepare_model_for_kbit_training(model)
qlora_model = get_peft_model(model, peft_config)


In [ ]:
qlora_model.print_trainable_parameters()

In [ ]:
import datetime

timestamp = datetime.datetime.now().strftime('%Y%m%d_%H%M%S')

demo_train = True
output_dir = f'models/llama-7-int4-dolly-{timestamp}'
print(output_dir)

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1 if demo_train else 3,
    max_steps=100,
    per_device_train_batch_size=3,
    gradient_accumulation_steps=1 if demo_train else 4,
    gradient_checkpointing=True,
    optim='paged_adamw_32bit',
    logging_steps=10,
    save_strategy='steps' if demo_train else 'epoch',
    save_steps=10,
    save_total_limit=2,
    learning_rate=2e-4,
    bf16=True,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type='constant'
)

In [ ]:
from trl import SFTTrainer

max_seq_length = 2048

trainer = SFTTrainer(
    model=qlora_model,
    train_dataset=dataset,
    peft_config=peft_config,
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    packing=True,
    formatting_func=format_instruction,
    args=args
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model()

## 使用微调后的 LLaMA2-7B 推理

In [ ]:
import torch
from peft import AutoPeftModelForCausalLM
from transformers import AutoTokenizer

model_dir = ''

model = AutoPeftModelForCausalLM.from_pretrained(
    model_dir,
    low_cpu_mem_usage=True,
    torch_dtype=torch.float16,
    load_in_4bit=True
)
tokenizer = AutoTokenizer.from_pretrained(model_dir)

In [ ]:
from datasets import load_dataset
from random import randrange

dataset = load_dataset("databricks/databricks-dolly-15k", split='train')
sample = dataset[randrange(len(dataset))]

prompt = f"""
### Instruction:
Use the Input below to create an instruction, which could have been used to generate the input using an LLM. 
 
### Input:
{sample['response']}
 
### Response:
""".strip()

input_ids = tokenizer(prompt, return_tensors='pt', truncation=True).input_ids.cuda()
outputs = model.generate(input_ids=input_ids, max_new_tokens=100, do_sample=True, top_p=0.9, temperature=0.9)

print(f'Prompt:\n{sample["response"]}\n')
print(f'Generated instruction:\n{tokenizer.batch_decode(outputs.detach().cpu().numpy(), skip_special_tokens=True)[0][len(prompt):]}')
print(f'Ground truth:\n{sample["instruction"]}')